# Introduction

In [9]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

# Getting info on various tables

In [10]:
%%sql
SELECT * FROM sqlite_master WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


# Accessing facts table

In [11]:
%%sql
SELECT * FROM facts LIMIT 5

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


## Checking outliers in population and population growth

In [12]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth) FROM facts

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


## Focussing on the outliers

In [13]:
%%sql
SELECT * FROM facts
WHERE population = (SELECT MIN(population) FROM facts)

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


#### The above country name Antarctica shows why the population can be 0

In [14]:
%%sql
SELECT * FROM facts
WHERE population = (SELECT MAX(population) FROM facts)

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


#### The above row represent the whole world as a country and hence it resulted into a population of 7.2 billion

# Calculating average for population and area

In [15]:
%%sql
SELECT AVG(population) avg_population, AVG(area) avg_area FROM facts

 * sqlite:///factbook.db
Done.


avg_population,avg_area
62094928.32231405,555093.546184739


# Finding countries which are densely populated

In [16]:
%%sql
SELECT * FROM facts
WHERE population > (SELECT AVG(population) From facts) and area < (select avg(area) from facts)

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
173,th,Thailand,513120,510890,2230,67976405,0.34,11.19,7.8,0.0
185,uk,United Kingdom,243610,241930,1680,64088222,0.54,12.17,9.35,2.54
192,vm,Vietnam,331210,310070,21140,94348835,0.97,15.96,5.93,0.3


# Countries having more water to land ratio

In [23]:
%%sql
select cast(area_water as float)/cast(area_land as float) as ratio from facts
where ratio > 0
order by ratio asc
limit 10

 * sqlite:///factbook.db
Done.


ratio
0.0001953621036591322
0.00023683587274019106
0.000560161326462021
0.0005697611479873187
0.0007833920877399138
0.0008628127696289905
0.0010796624808982792
0.0010821339681852614
0.001217068104799038
0.0014609203798392988


# Countries having more water than land

In [24]:
%%sql
select * from facts
where area_water > area_land

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
228,io,British Indian Ocean Territory,54400,60,54340,None,None,None,None,None
247,vq,Virgin Islands,1910,346,1564,103574,0.59,10.31,8.54,7.67


# Country that will add most population next year

In [25]:
%%sql
select name, MAX((population*population_growth)/100) from facts

 * sqlite:///factbook.db
Done.


name,MAX((population*population_growth)/100)
World,78370092.1188


## Haha off course how can a country beat the world. Excluding the world

In [26]:
%%sql
select name, MAX((population*population_growth)/100) from facts
where name != 'World'

 * sqlite:///factbook.db
Done.


name,MAX((population*population_growth)/100)
India,15270686.1248


### My goodness gracious. :((

# Countries having higher death_rate than birth_rate

In [27]:
%%sql
select * from facts
where death_rate > birth_rate

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56
16,bo,Belarus,207600,202900,4700,9589689,0.2,10.7,13.36,0.7
22,bk,Bosnia and Herzegovina,51197,51187,10,3867055,0.13,8.87,9.75,0.38
26,bu,Bulgaria,110879,108489,2390,7186893,0.58,8.92,14.44,0.29
44,hr,Croatia,56594,55974,620,4464844,0.13,9.45,12.18,1.39
47,ez,Czech Republic,78867,77247,1620,10644842,0.16,9.63,10.34,2.33
57,en,Estonia,45228,42388,2840,1265420,0.55,10.51,12.4,3.6
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
67,gr,Greece,131957,130647,1310,10775643,0.01,8.66,11.09,2.32
75,hu,Hungary,93028,89608,3420,9897541,0.22,9.16,12.73,1.33
